## The Data API

### Creating Dataset

In [4]:
import tensorflow as tf

X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [5]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### Chaining Transformation

In [6]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [7]:
dataset = dataset.map(lambda x: x*2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [8]:
dataset = dataset.apply(tf.data.experimental.unbatch())

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.


In [9]:
for item in dataset.take(5):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


### Shuffling the Data

In [10]:
tf.random.set_seed(42)
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 6 5 7 3 9], shape=(7,), dtype=int64)
tf.Tensor([8 2 1 0 4 6 4], shape=(7,), dtype=int64)
tf.Tensor([7 2 5 9 2 1 3], shape=(7,), dtype=int64)
tf.Tensor([4 3 8 7 9 5 0], shape=(7,), dtype=int64)
tf.Tensor([8 6], shape=(2,), dtype=int64)


In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), 
                                                    random_state=42)
scaler = StandardScaler()
scaler.fit(X_train)
X_mean, X_std = scaler.mean_, scaler.scale_

In [13]:
import os
import numpy as np

def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")
    
    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [14]:
train_data = np.c_[X_train, y_train]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header=header, n_parts=20)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header=header, n_parts=10)

In [16]:
import pandas as pd

pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,4.2143,37.0,5.288235,0.973529,860.0,2.529412,33.81,-118.12,2.285
1,5.3468,42.0,6.364322,1.087940,957.0,2.404523,37.16,-121.98,2.799
2,3.9191,36.0,6.110063,1.059748,711.0,2.235849,38.45,-122.69,1.830
3,6.3703,32.0,6.000000,0.990196,1159.0,2.272549,34.16,-118.41,4.658
4,2.3684,17.0,4.795858,1.035503,706.0,2.088757,38.57,-121.33,1.500


In [19]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
4.2143,37.0,5.288235294117647,0.9735294117647059,860.0,2.5294117647058822,33.81,-118.12,2.285
5.3468,42.0,6.364321608040201,1.0879396984924623,957.0,2.4045226130653266,37.16,-121.98,2.799
3.9191,36.0,6.110062893081761,1.059748427672956,711.0,2.2358490566037736,38.45,-122.69,1.83
6.3703,32.0,6.0,0.9901960784313726,1159.0,2.272549019607843,34.16,-118.41,4.658


### Building an Input Pipeline

In [21]:
import tensorflow as tf
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [22]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets\\housing\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_15.csv', sh

In [23]:
n_reader = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1), 
    cycle_length=n_reader
)

In [24]:
for line in dataset.take(5):
    print(line.numpy())

b'3.3125,11.0,5.361736334405145,1.0578778135048232,1963.0,3.1559485530546625,38.69,-121.46,0.968'
b'2.4524,41.0,5.340116279069767,1.188953488372093,1430.0,4.156976744186046,34.13,-117.32,0.704'
b'5.8735,35.0,5.811638591117918,1.0566615620214395,1521.0,2.329249617151608,34.11,-118.63,4.481'
b'3.8788,20.0,5.140068886337543,1.060849598163031,2656.0,3.049368541905855,37.13,-121.66,2.269'
b'4.5893,39.0,5.711688311688311,1.0077922077922077,1025.0,2.6623376623376624,37.95,-122.07,1.9'


### Processing the Data

In [25]:
n_inputs = 8

def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean)/X_std, y

In [26]:
preprocess(b'4.2143,37.0,5.288235294117647,0.9735294117647059,860.0,2.5294117647058822,33.81,-118.12,2.285')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.17648865,  0.66640687, -0.06085434, -0.2811182 , -0.49654418,
        -0.04828325, -0.86074144,  0.73099613], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.285], dtype=float32)>)

In [32]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5, 
                       n_read_threads=None, shuffle_buffer_size=10000, 
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), 
                                 cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

In [30]:
X_train[0]

array([   4.2143    ,   37.        ,    5.28823529,    0.97352941,
        860.        ,    2.52941176,   33.81      , -118.12      ])

### Using Dataset with tf.keras

In [33]:
train_set = csv_reader_dataset(train_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [78]:
from tensorflow import keras
from tensorflow.keras import optimizers, callbacks
model = keras.Sequential([keras.layers.Dense(100, activation="relu"), 
                          keras.layers.BatchNormalization(), 
                          keras.layers.Dense(200, activation="relu"), 
                          keras.layers.BatchNormalization(), 
                          keras.layers.Dense(10, activation="relu"), 
                          keras.layers.BatchNormalization(), 
                          keras.layers.Dense(1)])
model.compile(loss="mse", optimizer=optimizers.SGD(learning_rate=1e-2, nesterov=True, 
                                                   momentum=0.93), metrics=["mae"])
model.fit(train_set, epochs=100, 
          validation_data=test_set, 
          callbacks=[callbacks.EarlyStopping(patience=10, 
                                             restore_best_weights=True, 
                                             monitor="val_loss")])

Epoch 1/100
484/484 [==============================] - 2s 2ms/step - loss: 0.5514 - mae: 0.5358 - val_loss: 0.5612 - val_mae: 0.5202
Epoch 2/100
484/484 [==============================] - 1s 1ms/step - loss: 0.4150 - mae: 0.4621 - val_loss: 0.3551 - val_mae: 0.4313
Epoch 3/100
484/484 [==============================] - 1s 1ms/step - loss: 0.3963 - mae: 0.4492 - val_loss: 0.4187 - val_mae: 0.4794
Epoch 4/100
484/484 [==============================] - 1s 1ms/step - loss: 0.3832 - mae: 0.4426 - val_loss: 0.5200 - val_mae: 0.4968
Epoch 5/100
484/484 [==============================] - 1s 1ms/step - loss: 0.3792 - mae: 0.4359 - val_loss: 0.4806 - val_mae: 0.5197
Epoch 6/100
484/484 [==============================] - 1s 2ms/step - loss: 0.3643 - mae: 0.4304 - val_loss: 0.3219 - val_mae: 0.4059
Epoch 7/100
484/484 [==============================] - 1s 1ms/step - loss: 0.3563 - mae: 0.4240 - val_loss: 0.4373 - val_mae: 0.4445
Epoch 8/100
484/484 [==============================] - 1s 1ms/step - 

In [80]:
model.evaluate(test_set)

162/162 [==============================] - 0s 1ms/step - loss: 0.3219 - mae: 0.4059


[0.3218575417995453, 0.4059430658817291]

In [81]:
new_set = test_set.take(3).map(lambda X, y: X)
model.predict(new_set)

array([[2.9361138 ],
       [0.9133055 ],
       [2.434945  ],
       [2.3590877 ],
       [1.2241691 ],
       [2.530606  ],
       [4.6654253 ],
       [2.7495708 ],
       [0.93317914],
       [2.0079727 ],
       [3.4963975 ],
       [2.6451511 ],
       [2.0391731 ],
       [1.194759  ],
       [1.3651963 ],
       [5.110795  ],
       [3.0155265 ],
       [1.4997151 ],
       [2.5836954 ],
       [1.0702391 ],
       [2.1778834 ],
       [1.0713279 ],
       [0.9217081 ],
       [2.8020098 ],
       [1.7460865 ],
       [2.945178  ],
       [1.7114737 ],
       [2.0891716 ],
       [2.305393  ],
       [2.0068982 ],
       [1.7338572 ],
       [2.7331486 ],
       [3.7790947 ],
       [0.9678383 ],
       [2.425568  ],
       [2.6264007 ],
       [1.9635203 ],
       [1.2492391 ],
       [1.0608734 ],
       [2.624142  ],
       [2.179114  ],
       [1.9583938 ],
       [0.8439145 ],
       [1.301668  ],
       [1.7930489 ],
       [1.8987231 ],
       [1.1935368 ],
       [2.214

In [82]:
# Custom function to train model

@tf.function
def train(model, optimizers, loss_fn, n_epochs, **kwargs):
    train_set = csv_reader_dataset(train_filepaths, repeat=n_epochs)
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = loss_fn(y_batch, y_pred)
            loss = tf.add_n([main_loss] + model.losses)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    

## The TFRecord Format

### Creating a TFRecord dataset

In [1]:
import tensorflow as tf
from tensorflow import keras

with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And, This is the second Record.")
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And, This is the second Record.', shape=(), dtype=string)


### Compressed TFRecord Files

In [2]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed_data.tfrecord", options) as f:
    f.write("This is First One")
    f.write("And the Another One")
dataset = tf.data.TFRecordDataset(["my_compressed_data.tfrecord"], 
                                  compression_type="GZIP")
for item in dataset:
    print(item)

tf.Tensor(b'This is First One', shape=(), dtype=string)
tf.Tensor(b'And the Another One', shape=(), dtype=string)


### Protocol Buffer

In [3]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])), 
            "id": Feature(int64_list=Int64List(value=[123])), 
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", 
                                                          b"c@d.com"]))
        }))

In [4]:
with tf.io.TFRecordWriter("my_contact.tfrecord") as f:
    f.write(person_example.SerializeToString())

### Loading and Parsing Examples

In [15]:
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""), 
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0), 
    "emails": tf.io.VarLenFeature(tf.string),
}

for serialized_example in tf.data.TFRecordDataset(["my_contact.tfrecord"]):
    parsed_example = tf.io.parse_single_example(serialized_example, 
                                               feature_description)


In [16]:
tf.sparse.to_dense(parsed_example["emails"], default_value=b"")

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [17]:
parsed_example["emails"].values

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [18]:
dataset = tf.data.TFRecordDataset(["my_contact.tfrecord"]).batch(10)
for serialized_example in dataset:
    parsed_examples = tf.io.parse_example(serialized_example, 
                                          feature_description)

### Handling Lists of List Using SequenceExample ProtoBuf

In [21]:
FeatureList = tf.train.FeatureList
FeatureLists = tf.train.FeatureLists
SequenceExample = tf.train.SequenceExample

context = Features(feature={
    "author_id": Feature(int64_list=Int64List(value=[123])), 
    "title": Feature(bytes_list=BytesList(value=[b"A", b"desert", b"place", b"."])), 
    "pub_date": Feature(int64_list=Int64List(value=[1623, 12, 25]))
})

content = [["When", "shall", "we", "three", "meet", "again", "?"], 
           ["In", "thunder", ",", "lightning", ",", "or", "in", "rain", "?"]]
comments = [["When", "the", "hurlyburly", "'s", "done", "."], 
            ["When", "the", "battle", "'s", "lost", "and", "won", "."]]

def words_to_feature(words):
    return Feature(bytes_list=BytesList(value=[word.encode("utf-8") for word in words]))

content_features = [words_to_feature(sentence) for sentence in content]
comment_features = [words_to_feature(comment) for comment in comments]

sequence_example = SequenceExample(
    context=context, 
    feature_lists=FeatureLists(feature_list={
        "content": FeatureList(feature=content_features), 
        "comments": FeatureList(feature=comment_features)
    })
)

In [22]:
context_feature_descriptions = {
    "author_id": tf.io.FixedLenFeature([], tf.int64, default_value=0), 
    "title": tf.io.VarLenFeature(tf.string), 
    "pub_data": tf.io.FixedLenFeature([3], tf.int64, default_value=[0, 0, 0]), 
}

sequence_feature_descriptions = {
    "content": tf.io.VarLenFeature(tf.string), 
    "comments": tf.io.VarLenFeature(tf.string), 
}

serialized_sequence_example = sequence_example.SerializeToString()

In [25]:
parsed_context, parsed_feature_lists = tf.io.parse_single_sequence_example(
    serialized_sequence_example, context_feature_descriptions, 
    sequence_feature_descriptions)
parsed_content = tf.RaggedTensor.from_sparse(parsed_feature_lists["content"])

In [26]:
parsed_context

{'title': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x274a6b47af0>,
 'author_id': <tf.Tensor: shape=(), dtype=int64, numpy=123>,
 'pub_data': <tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 0, 0], dtype=int64)>}

In [27]:
parsed_context["title"].values

<tf.Tensor: shape=(4,), dtype=string, numpy=array([b'A', b'desert', b'place', b'.'], dtype=object)>

In [28]:
parsed_feature_lists

{'comments': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x274ade08160>,
 'content': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x274ae02c9d0>}

In [29]:
parsed_content

<tf.RaggedTensor [[b'When', b'shall', b'we', b'three', b'meet', b'again', b'?'], [b'In', b'thunder', b',', b'lightning', b',', b'or', b'in', b'rain', b'?']]>

## Preprocessing the Input Features